## Comparing EM with GD

### First, we set up by choosing the number of clusters $m$, the number of features $n$, and generate samples by use of a BMM with random parameters. The goal is to recover this setting. The sample set is $S$, which is a dictionary containing binary strings $x$. Notice that replicates are gathered together to save space.

In [12]:
import nbm
import numpy as np
m = 3  # number of classes
n = 5 # number of features
samples = 2**20 # sample size
# generate the parameters

### Try 200 iterations. For each iteration, generate samples; use EM to learn the parameters and compute the ratio; and use GD to find k-cluster points. Compute the ratios of the likelihoods. 

In [15]:
ratios = np.zeros(10)
count = 0
print("m, n:", m, n)
for __ in range(200):
    [gtheta, gPhi] = nbm.randInt(m, n)
    total = np.zeros(n)
    ct = 0
    # generate samples
    S = dict()
    for _ in range(2**n):    # the sample space
    # compute the probability
        x = nbm.decToBits(_, n)   # convert decimal to binary
        prob = 0.0
        for __ in range(m):
            prob += gtheta[__] * nbm.bern_n(x, gPhi[__, :], n) 
        S[_] = prob * samples
    # EM
    ell0 = nbm.ell(n, gPhi, gtheta, S, samples)
    [theta, Phi] = nbm.randInt(m, n)
    [theta1, Phi1] = nbm.em(n, m, samples, S, theta, Phi)
    ell1 = nbm.ell(n, Phi1, theta1, S, samples)
    # GD
    find_k = 0   # if a k-cluster point is found
    for _ in range(10):
        [theta, Phi] = nbm.randInt(m, n)
        [ell2, theta, Phi, iterId] = nbm.gradientDescent(n, samples, S, theta, Phi, numIterations=4000, alpha = 0.02, tolerance = 1e-4)
        if min(theta) < 0.0005:
            find_k = 1
            break
    # if a k-cluster point is found
    if find_k == 1:
        ratio = np.exp(ell1 - ell2)
        print("GD/EM: ", ratio, "count: ", count)
        print("GD/opt: ", np.exp(ell0 - ell2))
        print("EM/opt: ", np.exp(ell0 - ell1))
  #      [rand_theta, rand_Phi] = nbm.randInt(m, n)
  #      ell_rand = nbm.ell(n, rand_Phi, rand_theta, S, samples)
 #       print("rand/opt: ", np.exp(ell0 - ell_rand))
        ratios[count] = ratio
        count += 1
        print("count: ", count)
    if count == 10:    # if collected 10 points
        break

m, n: 3 5
GD converges at  445 steps
GD/EM:  0.9927920186089064 count:  0
GD/opt:  0.9927921609730024
EM/opt:  1.000000143397704
count:  1
GD converges at  399 steps
GD converges at  401 steps
GD converges at  849 steps
GD converges at  415 steps
GD/EM:  0.9983263397463682 count:  1
GD/opt:  0.9983263267510081
EM/opt:  0.9999999869828536
count:  2
GD converges at  1321 steps
GD converges at  664 steps
GD converges at  1005 steps
GD converges at  2677 steps
GD converges at  1893 steps
GD converges at  1684 steps
GD/EM:  0.9972603852179658 count:  2
GD/opt:  0.9972606180823889
EM/opt:  1.0000002335041345
count:  3
GD converges at  965 steps
GD converges at  792 steps
GD/EM:  0.9730725591661532 count:  3
GD/opt:  0.9730740252809377
EM/opt:  1.0000015066859822
count:  4
GD converges at  1011 steps
GD converges at  1136 steps
GD converges at  1452 steps
GD converges at  682 steps
GD converges at  938 steps
GD/EM:  0.9901704408600936 count:  4
GD/opt:  0.9901712907321689
EM/opt:  1.000000858

### Compute worst-case ratio and average ratio

In [20]:
print("number of cluster: ", m, "number of features: ", n)
print("min_ratio: ", min(ratios))
print("avg_ratio: ", np.sum(ratios)/10)

number of cluster:  3 number of features:  5
min_ratio:  0.8314761095775764
avg_ratio:  0.9728560152345611
